# Introdruction to Linked Open Data with DraCor

*Ingo Börner* (ingo.boerner@uni-potsdam.de)

## Example: Linked Data used on the DraCor platform
When you go to the detail view of a play on the DraCor frontend, you will notice, that for most of the plays we display an image of it's author next to the years of his or her birth an death. Here is an example of the play "Julius Ceasar" by William Shakespeare, taken from "shakedracor".


![View of 'Julius Ceasar' with Shakespeare's portrait](images/frontend_header_julius_ceasar.png "View of 'Julius Ceasar' with Shakespeare's portrait")

If you look into the `<teiHeader>` of the [underlying data](https://raw.githubusercontent.com/dracor-org/shakedracor/main/tei/julius-caesar.xml), you won't find this information encoded there. Here is how the author is encoded in the `<author>` element:

```
<author>
  <persName>
    <forename>William</forename>
    <surname>Shakespeare</surname>
  </persName>
  <idno type="wikidata">Q692</idno>
</author>
```

The element `<idno>` holds the identifer of the entity representing William Shakespeare on [Wikidata](https://www.wikidata.org) `Q692`: If you attach this number to `http://www.wikidata.org/entity/` you get the full URI, which is used to identify the entity http://www.wikidata.org/entity/Q692.

![William Shakespeare on Wikidata](images/shakespeare_wd_portrait.png "View of 'William Shakespeare on Wikidata")

![Birth an death dates of Shakespeare on Wikidata](images/shakespeare_wd_birth-death.png "Birth an death dates of Shakespeare on Wikidata")

This data is displayed on the frontend when you view the play on DraCor.

## Using Wikidata to retrieve additional information on a play with SPARQL

Apart from the author plays in DraCor, most of the plays also include the Wikidata identifier of the play itself. Here is an example of the play "The Waves of Sea and Love" of the Austrian author Franz Grillparzer:

![Grillparzer: Des Meeres und der Liebe Wellen](images/grillparzer_des_meeres_wellen.png "Grillparzer: Des Meeres und der Liebe Wellen")

You can find the Wikidata-Identifier `Q48929649` below the title. If you look at the page of this entity `http://www.wikidata.org/entity/Q48929649` [on Wikidata](https://www.wikidata.org/wiki/Q48929649), you can find out, that the play was staged at the "Burgtheater" in Vienna for the first time:

![Grillparzer: Des Meeres und der Liebe Wellen – location of first performance](images/location_of_first_performance_Grillparzer.png "Grillparzer: Des Meeres und der Liebe Wellen – Location of first performance")

Wikidata uses a so-called properties to store all kind of information. In this case it is the property `P4647` [location of first performance](https://www.wikidata.org/wiki/Property:P4647).

Using the [Wikidata Query Service](https://query.wikidata.org) and the query language **SPARQL** ([here](https://www.wikidata.org/wiki/Wikidata:SPARQL_tutorial) is a good tutorial) you can use these properties, e.g. to get the location of the first location of Grillparzer's play, we can insert the following query:

```
SELECT ?location ?locationLabel
WHERE 
{
  wd:Q48929649 wdt:P4647 ?location .
  SERVICE wikibase:label { bd:serviceParam wikibase:language "[AUTO_LANGUAGE],en". } 
}
```

Try it out yourself [here](https://query.wikidata.org/#SELECT%20%3Flocation%20%3FlocationLabel%0AWHERE%20%0A%7B%0A%20%20wd%3AQ48929649%20wdt%3AP4647%20%3Flocation%20.%0A%20%20SERVICE%20wikibase%3Alabel%20%7B%20bd%3AserviceParam%20wikibase%3Alanguage%20%22%5BAUTO_LANGUAGE%5D%2Cen%22.%20%7D%20%0A%7D).

You could also extend the query and get the coordinates of the theatre "Burgtheater" as well. You can make use of the query editor by starting to write `wdt:coor` and then press `CTRL` + space bar on your keyboard to get suggestions:

![ctrl + space to get suggestions](images/ctrl-space.png "ctrl + space to get suggestions")


Here is the SPARQL query:

```
SELECT ?location ?locationLabel ?coords
WHERE 
{
  wd:Q48929649 wdt:P4647 ?location .
  ?location wdt:P625 ?coords .
  SERVICE wikibase:label { bd:serviceParam wikibase:language "[AUTO_LANGUAGE],en". } 
}
```

Try it out [here](https://query.wikidata.org/#SELECT%20%3Flocation%20%3FlocationLabel%20%3Fcoords%0AWHERE%20%0A%7B%0A%20%20wd%3AQ48929649%20wdt%3AP4647%20%3Flocation%20.%0A%20%20%3Flocation%20wdt%3AP625%20%3Fcoords%20.%0A%20%20SERVICE%20wikibase%3Alabel%20%7B%20bd%3AserviceParam%20wikibase%3Alanguage%20%22%5BAUTO_LANGUAGE%5D%2Cen%22.%20%7D%20%0A%7D).

![Querying for the coordinates of the location of first performances](images/coordinates_first_performances_grillparzer.png "Querying for the coordinates of the location of first performances")

## Example: Locations of first performances on Wikidata

If you replace the identifier of the play by Grillparzer `wd:Q48929649` in the example with a variable, e.g. `?play` and modify the query a bit, you can query for the locations of first locations of plays in Wikidata:



```
SELECT ?play ?playLabel ?authorLabel ?locationLabel ?coords 
WHERE 
{
  ?play wdt:P7937 wd:Q25379 ;
         wdt:P50 ?author ;
         wdt:P4647 ?location .
  
  ?location wdt:P625 ?coords .
  
  SERVICE wikibase:label { bd:serviceParam wikibase:language "[AUTO_LANGUAGE],en". }
}
LIMIT 500
```

In the line `?play wdt:P7937 wd:Q25379` we limit our results to everything that is a play by making use of the property `P7937` [form of creative work](https://www.wikidata.org/wiki/Property:P7937) and the entity `wd:Q25379`, which stands for "play" (http://www.wikidata.org/entity/Q25379). Additionally, we want to get the play's author using the property `P50` [author](https://www.wikidata.org/wiki/Property:P50). And we also want to get the coordinates of the locations to visualize them on a map.

In the last line we `LIMIT` our query to 500 (random) results only to prevent the service from taking too long or crashing.

Try the quere [here](https://query.wikidata.org/#%23Dramen%0ASELECT%20%3Fplay%20%3FplayLabel%20%3FauthorLabel%20%3FlocationLabel%20%3Fcoords%20%0AWHERE%20%0A%7B%0A%20%20%3Fplay%20wdt%3AP7937%20wd%3AQ25379%20%3B%0A%20%20%20%20%20%20%20%20%20wdt%3AP50%20%3Fauthor%20%3B%0A%20%20%20%20%20%20%20%20%20wdt%3AP4647%20%3Flocation%20.%0A%20%20%0A%20%20%3Flocation%20wdt%3AP625%20%3Fcoords%20.%0A%20%20%0A%20%20SERVICE%20wikibase%3Alabel%20%7B%20bd%3AserviceParam%20wikibase%3Alanguage%20%22%5BAUTO_LANGUAGE%5D%2Cen%22.%20%7D%20%23%20%3Cspan%20lang%3D%22en%22%20dir%3D%22ltr%22%20class%3D%22mw-content-ltr%22%3EHelps%20get%20the%20label%20in%20your%20language%2C%20if%20not%2C%20then%20en%20language%3C%2Fspan%3E%0A%7D%0ALIMIT%20500). You can also switch to the map view.

Here is the result:

In [15]:
#run this cell to embed the result
from IPython.display import IFrame
IFrame(' https://query.wikidata.org/embed.html#%23Dramen%0ASELECT%20%3Fplay%20%3FplayLabel%20%3FauthorLabel%20%3FlocationLabel%20%3Fcoords%20%0AWHERE%20%0A%7B%0A%20%20%3Fplay%20wdt%3AP7937%20wd%3AQ25379%20%3B%0A%20%20%20%20%20%20%20%20%20wdt%3AP50%20%3Fauthor%20%3B%0A%20%20%20%20%20%20%20%20%20wdt%3AP4647%20%3Flocation%20.%0A%20%20%0A%20%20%3Flocation%20wdt%3AP625%20%3Fcoords%20.%0A%20%20%0A%20%20SERVICE%20wikibase%3Alabel%20%7B%20bd%3AserviceParam%20wikibase%3Alanguage%20%22%5BAUTO_LANGUAGE%5D%2Cen%22.%20%7D%0A%7D%0ALIMIT%20500', width=1000, height=600)


The query above lists everything that is considered a play and the results are not restricted to play in DraCor. But linked open data makes it possible to combine these two data sources, but to be able to do this, you will have to understand how DraCor's RDF data is structured.

## DraCor Ontology

A good start is to have a look at the DraCor onlology. Down below it is visualized with WebVowl: https://vowl.acdh.oeaw.ac.at/#iri=https://raw.githubusercontent.com/dracor-org/dracor-schema/ontology/ontology/dracor-ontology.xml

Run the following cell to display it here:

In [17]:
#Display the Ontology in an iframe
IFrame('https://vowl.acdh.oeaw.ac.at/#iri=https://raw.githubusercontent.com/dracor-org/dracor-schema/ontology/ontology/dracor-ontology.xml', width=1000, height=600)




As an example, we look at two classes and a property from the DraCor ontology:

Class `play`:
```
<owl:Class rdf:about="http://dracor.org/ontology#play">
  <rdfs:subClassOf rdf:resource="http://www.w3.org/2002/07/owl#Thing"/>
  <rdfs:label xml:lang="de">Stück</rdfs:label>
  <rdfs:label xml:lang="en">play</rdfs:label>
  <rdfs:label xml:lang="ru">пьеса</rdfs:label>
</owl:Class>
```

Class `corpus`:

```
<owl:Class rdf:about="http://dracor.org/ontology#corpus">
  <rdfs:subClassOf rdf:resource="http://www.w3.org/2002/07/owl#Thing"/>
  <rdfs:label xml:lang="de">Korpus</rdfs:label>
  <rdfs:label xml:lang="en">corpus</rdfs:label>
  <rdfs:label xml:lang="ru">корпус</rdfs:label>
</owl:Class>
```

Property `in_corpus`:

```
<owl:ObjectProperty rdf:about="http://dracor.org/ontology#in_corpus">
  <rdfs:domain rdf:resource="http://dracor.org/ontology#play"/>
  <rdfs:range rdf:resource="http://dracor.org/ontology#corpus"/>
  <rdfs:comment xml:lang="de">Gibt die Zugehörigkeit eines Stücks zu einem Korpus an.</rdfs:comment>
  <rdfs:label xml:lang="de">Korpus</rdfs:label>
  <rdfs:label xml:lang="en">Corpus</rdfs:label>
  <rdfs:label xml:lang="ru">корпус</rdfs:label>
</owl:ObjectProperty>
```

In the data each play is an instance of the class "play" and is contained in an instance of the class "corpus". The connection of plays and corpora are expressed with the property `in_corpus`. To record the information, that the play X is contained in the corpus Y can be expressed in **RDF** (the example is in the **Turtle notation**):

The following two statements instantiate a Grillparzer's play identified by the URI `https://dracor.org/entity/ger000010` as an instance of the class `play` and attach a human-readable `rdfs:label` to it:

```
<https://dracor.org/entity/ger000010> rdf:type <http://dracor.org/ontology#play> .
<https://dracor.org/entity/ger000010> rdfs:label "Grillparzer, Franz: Des Meeres und der Liebe Wellen. Trauerspiel in fünf Aufzügen" .
```

The following two statements instantiate the "German Drama Corpus" identified by the URI `https://dracor.org/entity/corpus/ger` by using a shortcut (`a` means `rdf:type`) as an instance of the class `corpus` and add a `rdfs:label`. To attach more properties to a object, you can use `;`.

```
<https://dracor.org/entity/corpus/ger> a <http://dracor.org/ontology#corpus"> ;
    rdfs:label "German Drama Corpus" .
```

To record the information, that the play contained in the corpus, the last exemplary triple linkes them with the property `in_corpus`:

```
<https://dracor.org/entity/ger000010> dracon:in_corpus <https://dracor.org/entity/corpus/ger> .
```

The RDF serialization of the play contains a lot more triples, but they are all structured in the same way. The RDF serialization of DraCor data is still under development. We don't use our own custom ontology only , but also reuse other Ontologies, like CIDOC-CRM and FRBRoo. You can download an RDF file for each play on the frontend in the "Download tab", e.g. https://dracor.org/ger/grillparzer-des-meeres-und-der-liebe-wellen#downloads.

## SPARQLing DraCor Data

After we have briefly touched upon how DraCor's RDF data is structured, let's make use of it and query it with the query language SPARQL. 
Similar to Wikidata DraCor has it's own SPARQL endpoint with a query interface in place:

![Link to YASGUI](images/sparql_link.png "Link to YASGUI")

For this tutorial, we use the staging instance, which can be found here: https://staging.dracor.org/sparql. Using this Interface, you can for example query for all the plays contained in the German Drama Corpus:


```
PREFIX rdfs: <http://www.w3.org/2000/01/rdf-schema#>
PREFIX urn: <http://fliqz.com/>
PREFIX dracon: <http://dracor.org/ontology#>
SELECT ?play ?label FROM <https://dracor.clscor.io> WHERE {
  ?play dracon:in_corpus <https://dracor.org/entity/corpus/ger> ;
        rdfs:label ?label .
}
```

Try it out [here](https://staging.dracor.org/sparql#query=%0APREFIX+rdfs%3A+%3Chttp%3A%2F%2Fwww.w3.org%2F2000%2F01%2Frdf-schema%23%3E%0APREFIX+urn%3A+%3Chttp%3A%2F%2Ffliqz.com%2F%3E%0APREFIX+dracon%3A+%3Chttp%3A%2F%2Fdracor.org%2Fontology%23%3E%0ASELECT+%3Fplay+%3Flabel+FROM+%3Chttps%3A%2F%2Fdracor.clscor.io%3E+WHERE+%7B%0A++%3Fplay+dracon%3Ain_corpus+%3Chttps%3A%2F%2Fdracor.org%2Fentity%2Fcorpus%2Fger%3E+%3B%0A++++++++rdfs%3Alabel+%3Flabel+.%0A%7D%0A&contentTypeConstruct=text%2Fturtle&contentTypeSelect=application%2Fsparql-results%2Bjson&endpoint=https%3A%2F%2Fstaging.dracor.org%2Ffuseki%2Fsparql&requestMethod=POST&tabTitle=List+GerDraCor+plays&headers=%7B%22Content-Type%22%3A%22application%2Fx-www-form-urlencoded%22%7D&outputFormat=table)


### List connections of plays to Wikidata
As demonstrated above, plays in DraCor are normally connected to it's representations on Wikidata. To get plays in a corpus and list the corresponding links to Wikidata, you have to take a detour. Here is the SPARQL query:

```
PREFIX owl: <http://www.w3.org/2002/07/owl#>
PREFIX frbroo: <http://iflastandards.info/ns/fr/frbr/frbroo/>
PREFIX crm: <http://www.cidoc-crm.org/cidoc-crm/>
PREFIX rdfs: <http://www.w3.org/2000/01/rdf-schema#>
PREFIX urn: <http://fliqz.com/>
PREFIX dracon: <http://dracor.org/ontology#>
SELECT ?play ?wd FROM <https://dracor.clscor.io> WHERE {
  ?play dracon:in_corpus <https://dracor.org/entity/corpus/ger> ;
        crm:P165_incorporates ?expression .
  ?expression frbroo:R40i_is_representative_expression_for ?work .
  
  ?work owl:sameAs ?wd .
}
LIMIT 500
```

Try it [here](https://staging.dracor.org/sparql#query=PREFIX+owl%3A+%3Chttp%3A%2F%2Fwww.w3.org%2F2002%2F07%2Fowl%23%3E%0APREFIX+frbroo%3A+%3Chttp%3A%2F%2Fiflastandards.info%2Fns%2Ffr%2Ffrbr%2Ffrbroo%2F%3E%0APREFIX+crm%3A+%3Chttp%3A%2F%2Fwww.cidoc-crm.org%2Fcidoc-crm%2F%3E%0APREFIX+rdfs%3A+%3Chttp%3A%2F%2Fwww.w3.org%2F2000%2F01%2Frdf-schema%23%3E%0APREFIX+urn%3A+%3Chttp%3A%2F%2Ffliqz.com%2F%3E%0APREFIX+dracon%3A+%3Chttp%3A%2F%2Fdracor.org%2Fontology%23%3E%0ASELECT+%3Fplay+%3Fwd+FROM+%3Chttps%3A%2F%2Fdracor.clscor.io%3E+WHERE+%7B%0A++%3Fplay+dracon%3Ain_corpus+%3Chttps%3A%2F%2Fdracor.org%2Fentity%2Fcorpus%2Fger%3E+%3B%0A++++++++crm%3AP165_incorporates+%3Fexpression+.%0A++%3Fexpression+frbroo%3AR40i_is_representative_expression_for+%3Fwork+.%0A++%0A++%3Fwork+owl%3AsameAs+%3Fwd+.%0A%7D%0ALIMIT+500%0A&contentTypeConstruct=text%2Fturtle&contentTypeSelect=application%2Fsparql-results%2Bjson&endpoint=https%3A%2F%2Fstaging.dracor.org%2Ffuseki%2Fsparql&requestMethod=POST&tabTitle=GerDraCor+plays+linking+to+Wikidata&headers=%7B%22Content-Type%22%3A%22application%2Fx-www-form-urlencoded%22%7D&outputFormat=table).

The link to Wikidata is not established by linking the instance of `dracon:play` to the Wikidata entity, but is done on the level of the respective `Work` from the FRBRoo ontology. The link between the DraCor play an it's Work is done via an representative Expression, that is incorporate in the play. So, it's a somewhat compicated path, but we can look at the SPARQL query bit by bit:

In addition to the DraCor Ontology there are additonal ontologies used. We have to declare prefixes for them:

```
PREFIX owl: <http://www.w3.org/2002/07/owl#>
PREFIX frbroo: <http://iflastandards.info/ns/fr/frbr/frbroo/>
PREFIX crm: <http://www.cidoc-crm.org/cidoc-crm/>
```

The first line of the `SELECT` clause should already look familiar: `?play dracon:in_corpus <https://dracor.org/entity/corpus/ger>` We are searching for plays in the GerDraCor corpus.

The next line uses a propery from CIDOC-CRM, to state the fact, that DraCor play, which could be understood as an Information Object (actually, a `subClass` of it, but nevermind..), which incorporates `crm:P165_incorporates` an "Expression" `F22_Self-Contained_Expression` (a class from the ontology FRBRoo). 

The statement `?expression frbroo:R40i_is_representative_expression_for ?work` says that this "Expression" is the `frbroo:R40i_is_representative_expression_for` of the Work in question.

Finally, the Work is the same `owl:sameAs` as the entity described by Wikidata.

### A federated query: Connecting DraCor and Wikidata
After we found a way to list the connections of DraCor to Wikidata, we can now make use of this information by writing a so-called federated query, which actually combines the information. Therefore we have to "call" an external endpoint, by using the `SERVICE` clause:

```
SERVICE <https://query.wikidata.org/sparql> {
  ?wd wdt:P4647 ?location ;
       rdfs:label ?wdLabel .

    ?location wdt:P625 ?coords ;
         rdfs:label ?locationLabel .

    FILTER (lang(?locationLabel) = "en")
    FILTER (lang(?wdLabel) = "en")
  }
```

This part of the query is somewhat identical to the one we executed to get the locations of first performances in Wikidata (see above), with the exception, that we use a variable `?wd` as a placeholder for the play. In our query, this variable will hold the URI of the Wikidata entity, that is linked to the Work of a respective DraCor play.

Here is the full query:

```
PREFIX owl: <http://www.w3.org/2002/07/owl#>
PREFIX frbroo: <http://iflastandards.info/ns/fr/frbr/frbroo/>
PREFIX crm: <http://www.cidoc-crm.org/cidoc-crm/>
PREFIX rdfs: <http://www.w3.org/2000/01/rdf-schema#>
PREFIX urn: <http://fliqz.com/>
PREFIX dracon: <http://dracor.org/ontology#>
PREFIX wd: <http://www.wikidata.org/entity/>
PREFIX wdt: <http://www.wikidata.org/prop/direct/>
SELECT ?play ?playLabel ?wd ?locationLabel ?coords FROM <https://dracor.clscor.io> WHERE {
  ?play a dracon:play ;
    	crm:P165_incorporates ?expression ;
        rdfs:label ?playLabel ;
        dracon:in_corpus <https://dracor.org/entity/corpus/ger> .

  ?expression a frbroo:F22_Self-Contained_Expression ;
              frbroo:R40i_is_representative_expression_for ?work .

  ?work owl:sameAs ?wd .

  SERVICE <https://query.wikidata.org/sparql> {
  ?wd wdt:P4647 ?location ;
       rdfs:label ?wdLabel .

    ?location wdt:P625 ?coords ;
         rdfs:label ?locationLabel .

    FILTER (lang(?locationLabel) = "en")
    FILTER (lang(?wdLabel) = "en")
  }

  FILTER(lang(?playLabel) = "ger")
}
LIMIT 100
```

Try it out [here](https://staging.dracor.org/sparql#query=PREFIX+owl%3A+%3Chttp%3A%2F%2Fwww.w3.org%2F2002%2F07%2Fowl%23%3E%0APREFIX+frbroo%3A+%3Chttp%3A%2F%2Fiflastandards.info%2Fns%2Ffr%2Ffrbr%2Ffrbroo%2F%3E%0APREFIX+crm%3A+%3Chttp%3A%2F%2Fwww.cidoc-crm.org%2Fcidoc-crm%2F%3E%0APREFIX+rdfs%3A+%3Chttp%3A%2F%2Fwww.w3.org%2F2000%2F01%2Frdf-schema%23%3E%0APREFIX+urn%3A+%3Chttp%3A%2F%2Ffliqz.com%2F%3E%0APREFIX+dracon%3A+%3Chttp%3A%2F%2Fdracor.org%2Fontology%23%3E%0APREFIX+wd%3A+%3Chttp%3A%2F%2Fwww.wikidata.org%2Fentity%2F%3E%0APREFIX+wdt%3A+%3Chttp%3A%2F%2Fwww.wikidata.org%2Fprop%2Fdirect%2F%3E%0ASELECT+%3Fplay+%3FplayLabel+%3Fwd+%3FlocationLabel+%3Fcoords+FROM+%3Chttps%3A%2F%2Fdracor.clscor.io%3E+WHERE+%7B%0A++%3Fplay+a+dracon%3Aplay+%3B%0A++++%09crm%3AP165_incorporates+%3Fexpression+%3B%0A++++++++rdfs%3Alabel+%3FplayLabel+%3B%0A++++++++dracon%3Ain_corpus+%3Chttps%3A%2F%2Fdracor.org%2Fentity%2Fcorpus%2Fger%3E+.%0A%0A++%3Fexpression+a+frbroo%3AF22_Self-Contained_Expression+%3B%0A++++++++++++++frbroo%3AR40i_is_representative_expression_for+%3Fwork+.%0A%0A++%3Fwork+owl%3AsameAs+%3Fwd+.%0A%0A++SERVICE+%3Chttps%3A%2F%2Fquery.wikidata.org%2Fsparql%3E+%7B%0A++%3Fwd+wdt%3AP4647+%3Flocation+%3B%0A+++++++rdfs%3Alabel+%3FwdLabel+.%0A%0A++++%3Flocation+wdt%3AP625+%3Fcoords+%3B%0A+++++++++rdfs%3Alabel+%3FlocationLabel+.%0A%0A++++FILTER+(lang(%3FlocationLabel)+%3D+%22en%22)%0A++++FILTER+(lang(%3FwdLabel)+%3D+%22en%22)%0A++%7D%0A%0A++FILTER(lang(%3FplayLabel)+%3D+%22ger%22)%0A%7D%0ALIMIT+100&contentTypeConstruct=text%2Fturtle&contentTypeSelect=application%2Fsparql-results%2Bjson&endpoint=https%3A%2F%2Fstaging.dracor.org%2Ffuseki%2Fsparql&requestMethod=POST&tabTitle=Locations+of+first+performances+Wikidata&headers=%7B%22Content-Type%22%3A%22application%2Fx-www-form-urlencoded%22%7D&outputFormat=leaflet)

![Federated Query visualized as map](images/federated_query_map.png "Federated query visualized as map")


### Characters across corpora
Another quick example to demonstrate what can be done: Here is query that lists characters of plays in all corpora, that are based on a mythological or historical figure by making use of the property `frbroo:R57_is_based_on` from the FRBRoo ontology:

```
PREFIX owl: <http://www.w3.org/2002/07/owl#>
PREFIX frbroo: <http://iflastandards.info/ns/fr/frbr/frbroo/>
PREFIX crm: <http://www.cidoc-crm.org/cidoc-crm/>
PREFIX rdfs: <http://www.w3.org/2000/01/rdf-schema#>
PREFIX urn: <http://fliqz.com/>
PREFIX dracon: <http://dracor.org/ontology#>
PREFIX wd: <http://www.wikidata.org/entity/>
PREFIX wdt: <http://www.wikidata.org/prop/direct/>
SELECT ?characterWD (SAMPLE(?characterLabel) AS ?characterLabelSample) (COUNT(?character) AS ?cntCharacters) (COUNT(DISTINCT ?corpus) as ?corpusCnt) FROM <https://dracor.clscor.io> WHERE {
  ?character a dracon:character ;
             rdfs:label ?characterLabel ;
             frbroo:R57_is_based_on ?characterWD ;
             dracon:is_character_in ?play .

  ?play dracon:in_corpus ?corpus .

}
GROUP BY ?characterWD
ORDER BY DESC(?cntCharacters)
```

Try it out [here](https://staging.dracor.org/sparql#query=PREFIX+owl%3A+%3Chttp%3A%2F%2Fwww.w3.org%2F2002%2F07%2Fowl%23%3E%0APREFIX+frbroo%3A+%3Chttp%3A%2F%2Fiflastandards.info%2Fns%2Ffr%2Ffrbr%2Ffrbroo%2F%3E%0APREFIX+crm%3A+%3Chttp%3A%2F%2Fwww.cidoc-crm.org%2Fcidoc-crm%2F%3E%0APREFIX+rdfs%3A+%3Chttp%3A%2F%2Fwww.w3.org%2F2000%2F01%2Frdf-schema%23%3E%0APREFIX+urn%3A+%3Chttp%3A%2F%2Ffliqz.com%2F%3E%0APREFIX+dracon%3A+%3Chttp%3A%2F%2Fdracor.org%2Fontology%23%3E%0APREFIX+wd%3A+%3Chttp%3A%2F%2Fwww.wikidata.org%2Fentity%2F%3E%0APREFIX+wdt%3A+%3Chttp%3A%2F%2Fwww.wikidata.org%2Fprop%2Fdirect%2F%3E%0ASELECT+%3FcharacterWD+(SAMPLE(%3FcharacterLabel)+AS+%3FcharacterLabelSample)+(COUNT(%3Fcharacter)+AS+%3FcntCharacters)+(COUNT(DISTINCT+%3Fcorpus)+as+%3FcorpusCnt)+FROM+%3Chttps%3A%2F%2Fdracor.clscor.io%3E+WHERE+%7B%0A++%3Fcharacter+a+dracon%3Acharacter+%3B%0A+++++++++++++rdfs%3Alabel+%3FcharacterLabel+%3B%0A+++++++++++++frbroo%3AR57_is_based_on+%3FcharacterWD+%3B%0A+++++++++++++dracon%3Ais_character_in+%3Fplay+.%0A%0A++%3Fplay+dracon%3Ain_corpus+%3Fcorpus+.%0A%0A%7D%0AGROUP+BY+%3FcharacterWD%0AORDER+BY+DESC(%3FcntCharacters)&contentTypeConstruct=text%2Fturtle&contentTypeSelect=application%2Fsparql-results%2Bjson&endpoint=https%3A%2F%2Fstaging.dracor.org%2Ffuseki%2Fsparql&requestMethod=POST&tabTitle=characters+across+corpora&headers=%7B%22Content-Type%22%3A%22application%2Fx-www-form-urlencoded%22%7D&outputFormat=table)